In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath("/netscratch/cunow/graph-matching"))

In [3]:
try:
  import torch_geometric
except:
  os.system("pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html")
  os.system("pip install torch-geometric")
    
try:
  import rdkit
except:
  os.system("pip install rdkit")

try:
  import pot
except:
  os.system("pip install pot")

try:
  import seaborn
except:
  os.system("pip install seaborn")
try:
  import wandb # need to do this before chaning CWD
except:
  os.system("pip install wandb")

Looking in indexes: http://pypi-cache/index, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


Looking in indexes: http://pypi-cache/index, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 345.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 368.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 336.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 299.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
import matplotlib.pylab as pl
import matplotlib.pyplot as plt

import gnn
from rdkit import Chem
#from rdkit.Chem import Draw
import utils
import util_metrics

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data,Batch
from torch_geometric.utils import to_dense_adj,to_dense_batch,get_embeddings
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
import vgae
import copy
import visualize
import wandb
import utils
import matching
from decoder import Decoder
import torch.nn.functional as F
import numpy as np
import gzip
import pickle
from tqdm import tqdm
print(wandb.__path__)

['/usr/local/lib/python3.10/dist-packages/wandb']


In [5]:
file_path='../data/QM9_canonical.pkl'
with open(file_path, 'rb') as file:
    dataset_smiles = pickle.load(file)

In [6]:
with gzip.open('../data/dataset.pkl.gz', 'rb') as f:
    dataset = pickle.load(f)

In [11]:
wandb.login(key="c4476f3ae06843fcc1472fbd0a5dadaceabd7557")
wandb.init(
    project="Graph Matching",
    config={
        "epochs": 500,
        "batch_size": 256,
        "encoder_norm":"graph",
        "lr": 5e-6,
        "dropout": 0.05,
        "sample_size":10000,
        "dataset_size": 129428,
        "latent_dim": 50
        })



wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


grad/decoder_gradients,█▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
grad/encoder_gradients,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
inference/Novelty,█▆▁▁▁▁▁▃▁▁▁▁▁▁▃▁▁▁▃▃▃▃▁▃▃▁▁▁▃▃▁▃▁▁▃▃▃▃▃▃
inference/Uniqueness,▁▂▅▅▆▆▄▅▅▅▅▆▅▄▅▅▅▅▃▄▃▄▅▄▄▄▄▄▄▄▄▄▄▆▄▇▇█▆▄
inference/Validity,▃██▃▁▁▁▂▂▂▃▄▄▅▅▆▆▆▆▆▇▇▇▇████████████████
train/kl_loss,█▆▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/matched_loss,███▇▇▂▂▂▁▂▂▁▁▂▂▁▁▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁
train/num_node_pred_loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/unmatched_loss,█▂▂▂▂▁▂▁▂▂▂▁▂▁▂▁▂▂▂▂▂▁▁▂▂▂▁▁▁▂▁▂▂▁▂▁▁▁▂▂
grad/decoder_gradients,0.07842
grad/encoder_gradients,0.00253


In [12]:
latent_dims=wandb.config["latent_dim"]
edge_dim=4
node_dim=4
train_loader=DataLoader(dataset,256,shuffle=True)
matcher=matching.NaiveMatcher()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder=gnn.GINE(in_channels=node_dim,hidden_channels=[16,32,64,128],out_channels=128,norm="graph",num_layers=4,
                  act="leakyrelu",edge_dim=4,dropout=0.1).to(device)
#decoder=gnn_decoder.Generator_Decoder(latent_dim=latent_dims,node_dim=node_dim,edge_dim=edge_dim,num_layers=4).to(device)
decoder=Decoder(latent_dims).to(device)
matcher=matching.NaiveMatcher()

vgae_model=vgae.VGAE(encoder=encoder,decoder=decoder,latent_dims=latent_dims,embedding="graph",eval=False).to(device)
#vgae_model.set_encoder()

In [ ]:
vgae_model.train()
optimizer = torch.optim.Adam(vgae_model.parameters(), lr=1e-5,weight_decay=1e-5)
kl_loss_ls,matched_loss_ls,unmatched_loss_ls,num_loss_ls=[],[],[],[]
alpha=0.01
beta=1.
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

for epoch in range(wandb.config["epochs"]):
    vgae_model.train()
    for data in tqdm(train_loader):
      data = data.to(device)
      optimizer.zero_grad()
      data_rec,mu,logvar,_,pred_num= vgae_model(data)
      num_nodes=torch.bincount(data.batch).to(torch.float32)
      num_loss=alpha*F.mse_loss(pred_num,num_nodes)

      matched_loss,unmatched_loss=matcher.diffusion_matching(data,data_rec,test=True)
      kl_loss=beta*vgae_model.kl_loss(mu,logvar)
      loss=matched_loss+kl_loss+num_loss
      loss.backward()
      encoder_grad,decoder_grad=compute_gradients(vgae_model)
      optimizer.step()
      #Log data and evaluate reconstruction quality
      #valid_rec,num_rec=util_metrics.evaluate_reconstruction(data,data_rec)
      wandb.log({"train/matched_loss":matched_loss,"train/unmatched_loss":unmatched_loss,
              "train/kl_loss":kl_loss,"train/num_node_pred_loss":num_loss})#,"valid_rec":valid_rec,"frac_rec":num_rec})
      wandb.log({"grad/encoder_gradients":encoder_grad,"grad/decoder_gradients":decoder_grad})
      optimizer.step()
    #Generate samples
    if epoch%5==0:
        path=f"../models/vgae_model_{epoch}.pth"
        torch.save(vgae_model.state_dict(), path)
    vgae_model.eval()
    sample=vgae_model.sample(wandb.config["sample_size"])    
    validity,unique,novelty,_=util_metrics.compute_metrics(sample,dataset_smiles)
    print(validity,unique,novelty)
    wandb.log({"inference/Validity":validity,"inference/Uniqueness":unique,"inference/Novelty":novelty})
wandb.finish()

100%|██████████| 506/506 [04:21<00:00,  1.93it/s]


0.9917 0.00010083694665725522 1.1102230246251565e-15


100%|██████████| 506/506 [04:23<00:00,  1.92it/s]


1.0 0.0001 1.1102230246251565e-15


100%|██████████| 506/506 [04:21<00:00,  1.93it/s]


...

100%|██████████| 506/506 [04:23<00:00,  1.92it/s]


0.9139 0.000547105810263705 2.220446049250313e-16


100%|██████████| 506/506 [04:21<00:00,  1.93it/s]


0.9202 0.0004346881112801565 2.220446049250313e-16


 43%|████▎     | 216/506 [01:52<02:29,  1.94it/s]

In [9]:
import torch

def compute_gradients(model):
    """
    Computes the gradients for the encoder and decoder separately and calculates their norms.

    Args:
        model: The VAE model containing encoder and decoder.

    Returns:
        encoder_gradients: Gradients of the encoder parameters.
        decoder_gradients: Gradients of the decoder parameters.
        encoder_norm: Norm of the encoder gradients.
        decoder_norm: Norm of the decoder gradients.
    """

    
    encoder_norm = 0.0
    decoder_norm = 0.0

    # Loop through model parameters
    for name, param in model.named_parameters():
        if param.grad is not None:  # Ensure the gradients exist
            if 'encoder' in name:

                encoder_norm += param.grad.data.norm(2).item() ** 2  # L2 norm
            elif 'decoder' in name:
                decoder_norm += param.grad.data.norm(2).item() ** 2  # L2 norm

    # Take the square root of the sums to get the final norms
    encoder_norm = encoder_norm ** 0.5
    decoder_norm = decoder_norm ** 0.5

    return encoder_norm, decoder_norm


In [ ]:
from torch_geometric.utils import  degree
DEVICE="cpu"
def dataset_to_degree_bin(train_dataset):
  """
  Convert a dataset to a histogram of node degrees (in-degrees).
  Load from file if available; otherwise, compute from the dataset.
  """



  # Compute the maximum in-degree in the training data.
  max_degree = -1
  for data in train_dataset:
    data = data
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    max_degree = max(max_degree, int(d.max()))

  # Create an empty histogram for degrees.
  deg = torch.zeros(max_degree + 1, dtype=torch.long, device=DEVICE)

  # Populate the histogram with data from the dataset.
  for data in train_dataset:
    data = data.to(DEVICE)
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())


  return deg

In [ ]:
dataset_to_degree_bin(dataset)

In [ ]:
torch.rand((8,6)).unsqueeze(-1)

In [ ]:
import torch

# Example tensor of shape (n, 9, 9, 4)
n = 3  # Example batch size
tensor = torch.rand((9, 9, 4))

# Create a new tensor with an additional 0 at the start of the fourth dimension
zeros = torch.zeros(( 9, 9, 1))
expanded_tensor = torch.cat((zeros, tensor), dim=2)

print(expanded_tensor.shape)  # Should be (n, 9, 9, 5)


Evaluate Model

In [ ]:
vgae_model.eval()
sample=vgae_model.sample(wandb.config["sample_size"])    
validity,unique,novelty,_=util_metrics.compute_metrics(sample,dataset_smiles)

In [ ]:
PATH="YOUR/PATH/HERE"
torch.save(vgae_model.state_dict(), PATH)

Generate Samples. Compute fraction of valid, unique, and novel molecules. Return SMILES representation of all novel SMILES.
Plot generated molecules

In [ ]:
vgae_model.eval()
sample=vgae_model.sample(1000)
validity,unique,novel,smiles_list=util_metrics.compute_metrics(sample,dataset_smiles)

In [ ]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem as Chem
mol_list = [Chem.MolFromSmiles(smiles) for smiles in list(smiles_list)]
Draw.MolsToGridImage(mol_list[:50], molsPerRow=10, subImgSize=(400,400),maxMols=100)